In [ ]:
# Metaheuristic algorithm applied to the Knapsack problem
# Author: Félix Watine
# Date: Feb 2024

import numpy as np
import random
import math


In [ ]:
# --- Knapsack Problem Functions ---

def knapsack_value(values, solution):
    """
    Calculate the total value of the knapsack based on the given solution.

    Parameters:
        values (array): The values of the items.
        solution (array): The solution vector (0 or 1 for each item).

    Returns:
        int: The total value of the knapsack.
    """
    return np.dot(values, solution)

def knapsack_weight(weights, solution):
    """
    Calculate the total weight of the knapsack based on the given solution.

    Parameters:
        weights (array): The weights of the items.
        solution (array): The solution vector (0 or 1 for each item).

    Returns:
        int: The total weight of the knapsack.
    """
    return np.dot(weights, solution)

def is_feasible(weights, solution, max_weight):
    """
    Check if a given solution is feasible based on the weight constraint.

    Parameters:
        weights (array): The weights of the items.
        solution (array): The solution vector (0 or 1 for each item).
        max_weight (int): The maximum allowable weight.

    Returns:
        bool: True if the solution is feasible, False otherwise.
    """
    return np.dot(weights, solution) <= max_weight


In [ ]:

# --- Simple Descent Method ---

def neighbors(solution, weights, max_weight):
    """
    Generate all feasible neighbors of a given solution.

    Parameters:
        solution (array): The current solution.
        weights (array): The weights of the items.
        max_weight (int): The maximum allowable weight.

    Returns:
        list: A list of feasible neighbor solutions.
    """
    n = len(solution)
    neighbors = []
    for i in range(n):
        neighbor = solution.copy()
        neighbor[i] = (solution[i] + 1) % 2
        if is_feasible(weights, neighbor, max_weight):
            neighbors.append(neighbor)
    return neighbors

def simple_descent(values, weights, max_weight, initial_solution):
    """
    Perform a simple descent optimization for the knapsack problem.

    Parameters:
        values (array): The values of the items.
        weights (array): The weights of the items.
        max_weight (int): The maximum allowable weight.
        initial_solution (array): The initial solution vector.

    Returns:
        tuple: A tuple containing the list of values and solutions over iterations.
    """
    solutions = [initial_solution]
    values_list = [knapsack_value(values, initial_solution)]
    
    while True:
        current_solution = solutions[-1]
        current_value = values_list[-1]
        
        best_neighbor_value = current_value
        best_neighbor = current_solution
        
        for neighbor in neighbors(current_solution, weights, max_weight):
            neighbor_value = knapsack_value(values, neighbor)
            if neighbor_value > best_neighbor_value:
                best_neighbor_value = neighbor_value
                best_neighbor = neighbor

        if best_neighbor_value <= current_value:
            break

        solutions.append(best_neighbor)
        values_list.append(best_neighbor_value)

    return values_list, solutions

# --- Simulated Annealing ---

def simulated_annealing(values, weights, max_weight, initial_solution, T=2, alpha=0.90, j_max=50):
    """
    Perform simulated annealing for the knapsack problem.

    Parameters:
        values (array): The values of the items.
        weights (array): The weights of the items.
        max_weight (int): The maximum allowable weight.
        initial_solution (array): The initial solution vector.
        T (float): Initial temperature.
        alpha (float): Cooling rate.
        j_max (int): Maximum number of iterations without improvement.

    Returns:
        tuple: A tuple containing the list of values, solutions, max value, and best solution.
    """
    solutions = [initial_solution]
    values_list = [knapsack_value(values, initial_solution)]
    max_value = values_list[0]
    best_solution = initial_solution
    i = 0
    
    while True:
        j = 0
        current_solution = solutions[-1]
        N = neighbors(current_solution, weights, max_weight)
        
        while True:
            k = random.randint(0, len(N) - 1)
            neighbor = N[k]
            delta_value = knapsack_value(values, neighbor) - values_list[-1]
            if random.uniform(0, 1) < np.exp(-delta_value / T):
                current_value = knapsack_value(values, neighbor)
                solutions.append(neighbor)
                values_list.append(current_value)
                break
            else:
                j += 1
                if j >= j_max:
                    return values_list, solutions, max_value, best_solution
        
        if values_list[-1] > max_value:
            max_value = values_list[-1]
            best_solution = solutions[-1]
        
        T *= alpha
        i += 1

# --- Tabu Search ---

def tabu_search(values, weights, max_weight, initial_solution, j_max=100):
    """
    Perform tabu search optimization for the knapsack problem.

    Parameters:
        values (array): The values of the items.
        weights (array): The weights of the items.
        max_weight (int): The maximum allowable weight.
        initial_solution (array): The initial solution vector.
        j_max (int): Maximum number of iterations without improvement.

    Returns:
        tuple: A tuple containing the list of values, solutions, max value, best solution, and tabu list.
    """
    solutions = [initial_solution]
    values_list = [knapsack_value(values, initial_solution)]
    tabu_list = []
    max_value = 0
    j = 0
    i = 0
    
    while True:
        current_solution = solutions[-1]
        N = neighbors(current_solution, weights, max_weight)
        
        if not N:
            break
        
        best_neighbor_value = 0
        best_neighbor = None
        
        for neighbor in N:
            if any(np.array_equal(neighbor, tabu_item) for tabu_item in tabu_list):
                continue
            neighbor_value = knapsack_value(values, neighbor)
            if neighbor_value > best_neighbor_value:
                best_neighbor_value = neighbor_value
                best_neighbor = neighbor
        
        if best_neighbor is None:
            break
        
        solutions.append(best_neighbor)
        values_list.append(best_neighbor_value)
        tabu_list.append(best_neighbor)
        
        if best_neighbor_value > max_value:
            max_value = best_neighbor_value
            best_solution = best_neighbor
        else:
            j += 1
            if j >= j_max:
                break

        i += 1

    return values_list, solutions, max_value, best_solution, tabu_list

# --- Ant Colony Optimization (ACO) ---

def greedy_force(values, weights):
    """
    Calculate the greedy force for each item based on its value-to-weight ratio.

    Parameters:
        values (array): The values of the items.
        weights (array): The weights of the items.

    Returns:
        array: An array representing the greedy force for each item.
    """
    n = len(values)
    force = np.ones((2, n))
    ratio = np.array(values) / np.array(weights)
    
    for i in range(n):
        force[1, i] = (ratio[i] - np.min(ratio)) / (np.max(ratio) - np.min(ratio))
        force[0, i] = 1 - force[1, i]
    
    return force

def generate_solutions(size, trace, force, alpha, beta, weights, max_weight):
    """
    Generate a population of feasible solutions using the ACO strategy.

    Parameters:
        size (int): The number of solutions to generate.
        trace (array): The trace matrix (pheromone levels).
        force (array): The greedy force matrix.
        alpha (float): The influence of the force.
        beta (float): The influence of the trace.
        weights (array): The weights of the items.
        max_weight (int): The maximum allowable weight.

    Returns:
        list: A list of feasible solutions.
    """
    n = trace.shape[1]
    probabilities = np.power(force, alpha) * np.power(trace, beta)
    probabilities /= np.sum(probabilities, axis=0)
    values = [0, 1]
    solutions = []
    
    while len(solutions) < size:
        solution = np.zeros(n, dtype=int)
        for j in range(n):
            solution[j] = np.random.choice(values, p=probabilities[:, j].ravel())
        
        if is_feasible(weights, solution, max_weight) and list(solution) not in solutions:
            solutions.append(solution)
    
    return solutions

def update_trace(trace, generation, decay=0.1, c=1):
    """
    Update the trace matrix (pheromone levels) based on the current generation of solutions.

    Parameters:
        trace (array): The current trace matrix.
        generation (list): The current generation of solutions.
        decay (float): The decay rate of the pheromone.
        c (float): A constant for updating the trace.

    Returns:
        array: The updated trace matrix.
    """
    delta = np.zeros(trace.shape)
    for solution in generation:
        for i in range(len(solution)):
            delta[solution[i], i] += knapsack_value(values, solution)
    
    trace = (1 - decay) * trace + c * delta
    return trace

def best_solution_in_population(population):
    """
    Identify the best solution in a given population.

    Parameters:
        population (list): A list of solutions.

    Returns:
        tuple: The best solution and its corresponding value.
    """
    max_value = 0
    best_solution = None
    
    for solution in population:
        value = knapsack_value(values, solution)
        if value > max_value:
            max_value = value
            best_solution = solution
    
    return max_value, best_solution

def ant_colony_optimization(values, weights, max_weight, initial_solution, j_max=50, population_size=5, alpha=0.5, beta=0.5):
    """
    Perform Ant Colony Optimization (ACO) for the knapsack problem.

    Parameters:
        values (array): The values of the items.
        weights (array): The weights of the items.
        max_weight (int): The maximum allowable weight.
        initial_solution (array): The initial solution vector.
        j_max (int): Maximum number of iterations without improvement.
        population_size (int): The number of solutions in each generation.
        alpha (float): The influence of the force.
        beta (float): The influence of the trace.

    Returns:
        tuple: The best value, best solution, last generation of solutions, trace matrix, and force matrix.
    """
    trace = np.ones((2, len(initial_solution)))
    force = greedy_force(values, weights)
    max_value = knapsack_value(values, initial_solution)
    best_solution = initial_solution
    
    for _ in range(j_max):
        generation = generate_solutions(population_size, trace, force, alpha, beta, weights, max_weight)
        trace = update_trace(trace, generation, c=1/population_size)
        current_value, current_solution = best_solution_in_population(generation)
        
        if current_value > max_value:
            max_value = current_value
            best_solution = current_solution

    return max_value, best_solution, generation, trace, force



In [ ]:
# Set the random seed for reproducibility
np.random.seed(42)

# Parameters
NUM_ITEMS = 100
W_MAX = 500

# Generate random values and weights for items
values = np.random.randint(1, 100, size=NUM_ITEMS)
weights = np.random.randint(1, 100, size=NUM_ITEMS)
initial_solution = np.zeros(NUM_ITEMS)

# --- Execution of Algorithms ---

# Simple Descent
values_list, solutions = simple_descent(values, weights, W_MAX, initial_solution)
print(f"Simple Descent - Best Value: {values_list[-1]}")
print(f"Simple Descent - Best Solution: {solutions[-1]}")

# Simulated Annealing
values_list, solutions, max_value, best_solution = simulated_annealing(values, weights, W_MAX, initial_solution)
print(f"Simulated Annealing - Best Value: {max_value}")
print(f"Simulated Annealing - Best Solution: {best_solution}")

# Tabu Search
values_list, solutions, max_value, best_solution, tabu_list = tabu_search(values, weights, W_MAX, initial_solution)
print(f"Tabu Search - Best Value: {max_value}")
print(f"Tabu Search - Best Solution: {best_solution}")
print(f"Tabu Search - Number of Tabu Solutions: {len(tabu_list)}")

# Ant Colony Optimization
max_value, best_solution, last_generation, trace, force = ant_colony_optimization(values, weights, W_MAX, initial_solution)
print(f"Ant Colony Optimization - Best Value: {max_value}")
print(f"Ant Colony Optimization - Best Solution: {best_solution}")
print(f"Ant Colony Optimization - Last Generation: {last_generation}")
print(f"Ant Colony Optimization - Trace: {trace}")
print(f"Ant Colony Optimization - Force: {force}")
